In [1]:
import numpy as np
import pandas as pd
import statsmodels.api as sm

# Example data
np.random.seed(42)
num_samples = 100
X = np.random.rand(num_samples, 5)
Y = 2 * X[:, 0] + 3 * np.sin(X[:, 1]) + 4 * np.log(X[:, 2]) + np.random.normal(0, 0.5, num_samples)

# Convert to DataFrame
data = pd.DataFrame(X, columns=['X1', 'X2', 'X3', 'X4', 'X5'])
data['Y'] = Y

# Create B-spline basis functions for all predictors
knots = [3, 5, 7]  # Adjust the number of knots as needed

# Initialize an empty list to store spline basis functions for each predictor
splines_list = []

# Loop over each predictor and create spline basis functions
for predictor in data.columns[:-1]:  # Exclude the last column which is the target variable
    splines = [sm.nonparametric.bspline(data[predictor], df=k, degree=3) for k in knots]
    splines_list.extend(splines)

# Combine all sets of spline basis functions and add a constant for the intercept
X_bspline = np.column_stack([spl.evaluated(data[predictor]) for spl in splines_list])
X_bspline = sm.add_constant(X_bspline)

# Fit the spline regression model
model = sm.OLS(data['Y'], X_bspline).fit()

# Perform hypothesis tests for individual coefficients
print("Hypothesis tests for individual coefficients:")
for predictor in data.columns[:-1]:
    print(f"Hypothesis test for {predictor}:")
    print(model.t_test(predictor))

# Perform F-test for joint significance of spline terms associated with each predictor
print("F-tests for joint significance of spline terms:")
hypotheses_list = [f"{predictor}={', '.join([f'{predictor}_{i}' for i in range(len(knots) + 3)])}" for predictor in data.columns[:-1]]
for i, predictor in enumerate(data.columns[:-1]):
    print(f"F-test for joint significance of spline terms associated with {predictor}:")
    print(model.f_test(hypotheses_list[i]))


AttributeError: module 'statsmodels.nonparametric.api' has no attribute 'bspline'

In [ ]:
from patsy import dmatrix

# Choose the number of knots for each feature
num_knots = 3

# Create the formula for spline regression using all columns except the dependent variable 'y'
formula = "SalePrice ~ " + " + ".join([f"bs({col}, knots={num_knots})" for col in X_train.columns if col != 'SalePrice'])

# Create spline basis matrix using patsy
spline_basis = dmatrix(formula, data=X_train)

# Fit the spline regression model
model = sm.OLS(y_train, spline_basis)
result = model.fit()

# Predict and plot the spline curve
pred_data = {col: np.linspace(X_train[col].min(), X_train[col].max(), 100) for col in X_train.columns if col != 'y'}
pred_df = pd.DataFrame(pred_data)
spline_basis_pred = dmatrix(formula, data=pred_df)
pred_y = result.predict(spline_basis_pred)

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# If the dataset has more than one independent variable, use a 3D plot
if len(X_train.columns) > 2:
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    ax.scatter(X_train.iloc[:, 0], X_train.iloc[:, 1], X_train.iloc[:, 2], c='blue', label='Data')
    ax.plot3D(pred_df.iloc[:, 0], pred_df.iloc[:, 1], pred_df.iloc[:, 2], color='red', label='Spline Regression')
    ax.set_xlabel(df.columns[0])
    ax.set_ylabel(df.columns[1])
    ax.set_zlabel(df.columns[2])
    ax.set_title('Spline Regression in 3D')
    plt.legend()
    plt.show()
# If the dataset has two independent variables, use a 2D plot
elif len(df.columns) == 2:
    plt.scatter(df.iloc[:, 0], df.iloc[:, 1], c='blue', label='Data')
    plt.plot(pred_df.iloc[:, 0], pred_df.iloc[:, 1], color='red', label='Spline Regression')
    plt.xlabel(df.columns[0])
    plt.ylabel(df.columns[1])
    plt.title('Spline Regression in 2D')
    plt.legend()
    plt.show()
# If the dataset has only one independent variable, use a 1D plot
else:
    plt.scatter(df.iloc[:, 0], df['y'], c='blue', label='Data')
    plt.plot(pred_df.iloc[:, 0], pred_y, color='red', label='Spline Regression')
    plt.xlabel(df.columns[0])
    plt.ylabel('y')
    plt.title('Spline Regression in 1D')
    plt.legend()
    plt.show()






In [ ]:
# Get predicted values
y_pred = trained_model.predict()

# Perform the Ramsey RESET test with a quadratic specification
X_quad = X_train.apply(np.square)
X_quad_train = pd.concat([X_train, X_quad], axis=1)

model_quad = sm.OLS(y_train, X_quad_train).fit()
y_pred_quad = model_quad.predict()

# Calculate the test statistic
reset_test_stat = ((y_pred_quad - y_pred) ** 2).sum()

# Perform F-test (assuming you have n observations and k predictors in total, including the intercept)
n = len(X_train)
k = X_train.shape[1] - 1  # Exclude the intercept
df1 = 2  # Degrees of freedom for the numerator (difference in degrees of freedom between models)
df2 = n - k - 1  # Degrees of freedom for the denominator

F_stat = (reset_test_stat / df1) / ((y - y_pred_quad).var() / df2)

# Perform the test by calculating the p-value (assuming you have scipy installed)
from scipy.stats import f

p_value = 1 - f.cdf(F_stat, df1, df2)

print("Ramsey RESET test statistic:", reset_test_stat)
print("F-statistic:", F_stat)
print("p-value:", p_value)